In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
X = pd.read_csv('../glass_formula_c.csv')

df = pd.read_csv('../glass.csv', sep=",")
Y = df['Type']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)
X

RI     Na    Mg    Al     Si     K    Ca    Ba   Fe      RI*Na  ...  \
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0  20.746576  ...   
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0  21.079603  ...   
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0  20.513915  ...   
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0  20.048289  ...   
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0  20.136163  ...   
..       ...    ...   ...   ...    ...   ...   ...   ...  ...        ...  ...   
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0  21.439492  ...   
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0  22.631402  ...   
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0  21.836534  ...   
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0  21.807414  ...   
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0  21.588475  ...   

        Si*K     Si*Ca     Si*Ba  Si*Fe    K*Ca    K*Ba  K*Fe    Ca*Ba  Ca*Fe  \
0     4.3068  628.0750    0.0000    0.0  0.5250  0.0000   0.0   0.0000    0.0   
1    34.9104  569.4759    0.0000    0.0  3.7584  0.0000   0.0   0.0000    0.0   
2    28.4661  567.8622    0.0000    0.0  3.0342  0.0000   0.0   0.0000    0.0   
3    41.3877  596.8542    0.0000    0.0  4.6854  0.0000   0.0   0.0000    0.0   
4    40.1940  589.7556    0.0000    0.0  4.4385  0.0000   0.0   0.0000    0.0   
..       ...       ...       ...    ...     ...     ...   ...      ...    ...   
209   5.8088  666.5598   76.9666    0.0  0.7344  0.0848   0.0   9.7308    0.0   
210   0.0000  613.7040  116.1654    0.0  0.0000  0.0000   0.0  13.3560    0.0   
211   0.0000  619.6648  120.4088    0.0  0.0000  0.0000   0.0  13.8416    0.0   
212   0.0000  624.2128  115.5677    0.0  0.0000  0.0000   0.0  13.3136    0.0   
213   0.0000  632.3632  122.5112    0.0  0.0000  0.0000   0.0  14.3954    0.0   

     Ba*Fe  
0      0.0  
1      0.0  
2      0.0  
3      0.0  
4      0.0  
..     ...  
209    0.0  
210    0.0  
211    0.0  
212    0.0  
213    0.0  

[214 rows x 45 columns]

In [4]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# KNN

In [5]:
%%time
plain_sfs = SFS(KNeighborsClassifier(), 
          k_features=(1, 20), 
          forward=True, 
          floating=False,
          scoring='accuracy', 
          cv=10)

plain_sfs.fit(X, Y)

CPU times: user 25.8 s, sys: 146 ms, total: 26 s
Wall time: 13 s


SequentialFeatureSelector(cv=10, estimator=KNeighborsClassifier(),
                          k_features=(1, 20), scoring='accuracy')

In [6]:
plain_sfs.k_score_

0.7199134199134198

In [7]:
selected_features = X.columns[list(plain_sfs.k_feature_idx_)]
x_t = X_train[selected_features]
clf = KNeighborsClassifier()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.62      0.89      0.73         9
           2       0.67      0.67      0.67        12
           3       0.00      0.00      0.00         3
           5       0.33      0.50      0.40         2
           6       0.00      0.00      0.00         2
           7       1.00      0.80      0.89         5

    accuracy                           0.64        33
   macro avg       0.44      0.48      0.45        33
weighted avg       0.58      0.64      0.60        33

0.6684210526315789


# Naive Bayes

In [8]:
naive_bayes = GaussianNB()
plain_sfs_nb = SFS(naive_bayes, 
          k_features=(1, 20), 
          forward=True, 
          floating=False, 
          cv=10)

plain_sfs_nb.fit(X, Y) 

SequentialFeatureSelector(cv=10, estimator=GaussianNB(), k_features=(1, 20))

In [9]:
plain_sfs_nb.k_score_

0.6170995670995671

In [10]:
selected_features = X.columns[list(plain_sfs_nb.k_feature_idx_)]
x_t = X_train[selected_features]
clf = GaussianNB()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.64      0.78      0.70         9
           2       0.77      0.83      0.80        12
           3       0.00      0.00      0.00         3
           5       1.00      1.00      1.00         2
           6       0.00      0.00      0.00         2
           7       0.83      1.00      0.91         5

    accuracy                           0.73        33
   macro avg       0.54      0.60      0.57        33
weighted avg       0.64      0.73      0.68        33

0.38713450292397666


# SVM

In [11]:

plain_sfs_svm = SFS(svm.SVC(), 
          k_features=(1, 20), 
          forward=True, 
          floating=False, 
          cv=10)

plain_sfs_svm.fit(X, Y)

SequentialFeatureSelector(cv=10, estimator=SVC(), k_features=(1, 20))

In [12]:
plain_sfs_svm.k_score_

0.6409090909090909

In [13]:
selected_features = X.columns[list(plain_sfs_svm.k_feature_idx_)]
x_t = X_train[selected_features]
clf = svm.SVC()
clf.fit(x_t, y_train)

x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.70      0.78      0.74         9
           2       0.69      0.92      0.79        12
           3       0.00      0.00      0.00         3
           5       1.00      0.50      0.67         2
           6       0.00      0.00      0.00         2
           7       0.67      0.80      0.73         5

    accuracy                           0.70        33
   macro avg       0.51      0.50      0.49        33
weighted avg       0.60      0.70      0.64        33

0.4421052631578948


# Logistic Regression

In [14]:
plain_sfs_lr = SFS(LogisticRegression(), 
          k_features=(1, 20), 
          forward=True, 
          floating=False, 
          cv=10)

plain_sfs_lr.fit(X, Y)
selected_features = X.columns[list(plain_sfs_lr.k_feature_idx_)]
x_t = X_train[selected_features]
clf = LogisticRegression()
clf.fit(x_t, y_train)
x_test_filtered = X_test[selected_features]
y_test_pred = clf.predict(x_test_filtered)
print(classification_report(y_test, y_test_pred))
print(cross_val_score(clf, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.78      0.78      0.78         9
           2       0.69      0.75      0.72        12
           3       0.00      0.00      0.00         3
           5       0.67      1.00      0.80         2
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         5

    accuracy                           0.76        33
   macro avg       0.63      0.75      0.68        33
weighted avg       0.70      0.76      0.72        33

0.6242690058479532
